In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import myfunctions as mf

In [ ]:
# =========================
# User-defined metadata
# =========================

varname = "tos"

# =========================
# Base CEDA paths
# =========================

CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

In [ ]:
#Model Names : IMPORTANT NOTE : grid descriptions NOT SAME as atmospheric data
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    # "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gn",},
    # "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    # "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}


In [ ]:
#Experiment details
EXPERIMENTS = {
    "piControl":     {"project": "CMIP",        "scenario": "piControl", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}

In [ ]:
for model_name, model_meta in MODELS.items():

    var = {}
    var_by_year = {}

    # --- LOAD DATA FOR A MODEL ---
    for exp, meta in EXPERIMENTS.items():
        if exp not in ["SSP245", "G6sulfur", "G6solar", "piControl"]:
            continue

        # open dataset
        # --- special-case ensemble override ---
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ensemble = "r1i1p1f2"
            else:
                ensemble = "r1i1p1f1"
        else:
            ensemble = model_meta["ensemble"]


        base = (
            CEDA_BASE
            / meta["project"]
            / model_meta["institution"]
            / model_name
            / meta["scenario"]
            / ensemble
            / "Omon"
            / varname
            / model_meta["grid"]
            / "latest"
        )

        print(str(base))
        # ds = mf.open_files(str(base))
        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ds = mf.open_files_CESM_G6sulfur(base)
            elif meta["scenario"] == "ssp585":
                ds = mf.open_files_CESM_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        elif model_name == "IPSL-CM6A-LR":
            if meta["scenario"] == "ssp585":
                ds = mf.open_files_IPSL_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        else:
            ds = mf.open_files(str(base))



        
        # ds = ...
        var[exp] = mf.read_var(ds, varname)

    # --- ANNUAL MEAN FOR THIS MODEL ---
    for exp, da in var.items():
        var_by_year[exp] = {
            "ANN": mf.seasonal_mean_by_year(da, 1, 12)
        }
        # londim = mf.get_lon_dim(da)

    # --- 2071–2100 MEAN FOR THIS MODEL --- : taking 30 years
    ts_mean = {}
    for exp in ["SSP245", "G6sulfur", "G6solar"]:
        ts_mean[exp] = (
            var_by_year[exp]["ANN"]
            .sel(year=slice(2071, 2101))
            .mean(dim="year")
        )

    # --- DIFFERENCES (MODEL-SPECIFIC) ---
    ts_mean["G6sulfur-SSP245"] = ts_mean["G6sulfur"] - ts_mean["SSP245"]
    ts_mean["G6solar-SSP245"]  = ts_mean["G6solar"]  - ts_mean["SSP245"]
    ts_mean["G6solar-G6sulfur"] = ts_mean["G6solar"] - ts_mean["G6sulfur"]

    # --- piControl internal variability ---
    if "piControl" in var_by_year:
    
        pi_ann = var_by_year["piControl"]["ANN"]
    
        # ensure full 30-year blocks only
        n_years = pi_ann.sizes["year"]
        n_blocks = n_years // 30
    
        # trim excess years at the end
        pi_ann_trim = pi_ann.isel(year=slice(0, n_blocks * 30))
    
        # create a block index: 0,0,...,1,1,...,2,2,...
        block = xr.DataArray(
            np.repeat(np.arange(n_blocks), 30),
            dims="year",
            coords={"year": pi_ann_trim.year},
            name="block"
        )
    
        # 30-year means
        pi_30yr_means = (
            pi_ann_trim
            .groupby(block)
            .mean(dim="year")
        )
    
        # std dev across 30-year means (gridpoint-wise)
        ts_variability = pi_30yr_means.std(dim="block")
    
    else:
        ts_variability = None

    

    # STORE RESULTS FOR ONE MODEL ONLY
    model_meta["ts_mean"] = ts_mean
    model_meta["ts_variability"] = ts_variability
    
    # #Verification
    # print(model_name, ts_mean["SSP245"].mean().values)

    # if ts_variability is not None:
    #     print(model_name, ts_variability.mean().values)


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

years_slice = slice(2081, 2101)  # 2081-2100
SCENARIOS = ["SSP245", "G6sulfur", "G6solar"]
DIFFS = [("G6sulfur", "SSP245"), ("G6solar", "SSP245"), ("G6solar", "G6sulfur")]

n_models = len(MODELS)+1
n_rows = 4
n_cols = n_models

fig, axes = plt.subplots(
    nrows=n_rows,
    ncols=n_cols,
    figsize=(2* 4 * n_cols, 3 * n_rows),
    subplot_kw={"projection": ccrs.PlateCarree(central_longitude=180)},
)

axes = np.atleast_2d(axes)

ssp245_vmin = 20
ssp245_vmax = 33

g6sulfur_vmax = g6solar_vmax = g6solar_g6sulfur_vmax = 1

# --- plotting ---
for col_idx, (model_name, model_meta) in enumerate(MODELS.items()):
    ts_mean = model_meta["ts_mean"]
    ts_var  = model_meta["ts_variability"]

    row_axes = axes[:, col_idx]

    # Row 0: SSP245
    im0 = row_axes[0].pcolormesh(
        ts_mean["SSP245"].longitude,
        ts_mean["SSP245"].latitude,
        ts_mean["SSP245"],
        vmin=ssp245_vmin,
        vmax=ssp245_vmax,
        cmap="jet",
        shading="auto",
        transform=ccrs.PlateCarree()
    )
    
    row_axes[0].add_feature(cfeature.LAND,facecolor="white",zorder=10)
    row_axes[0].coastlines()
    row_axes[0].set_title(f"{model_name} SSP245", fontsize=10)

    # Row 1: G6sulfur - SSP245
    im1 = row_axes[1].pcolormesh(
        ts_mean["G6sulfur-SSP245"].longitude,
        ts_mean["G6sulfur-SSP245"].latitude,
        ts_mean["G6sulfur-SSP245"],
        vmin=-g6sulfur_vmax,
        vmax=g6sulfur_vmax,
        cmap="bwr",
        shading="auto",
        transform=ccrs.PlateCarree()
    )

    row_axes[1].add_feature(cfeature.LAND,facecolor="white",zorder=10)
    row_axes[1].coastlines()
    row_axes[1].set_title("G6sulfur − SSP245 (Sig is Hashed)", fontsize=10)

    # Row 2: G6solar - SSP245
    im2 = row_axes[2].pcolormesh(
        ts_mean["G6solar-SSP245"].longitude,
        ts_mean["G6solar-SSP245"].latitude,
        ts_mean["G6solar-SSP245"],
        vmin=-g6solar_vmax,
        vmax=g6solar_vmax,
        cmap="bwr",
        shading="auto",
        transform=ccrs.PlateCarree()
    )

    row_axes[2].add_feature(cfeature.LAND,facecolor="white",zorder=10)
    row_axes[2].coastlines()
    row_axes[2].set_title("G6solar − SSP245 (Non-Sig is Hashed)", fontsize=10)

    # Row 3: G6solar - G6sulfur
    im3 = row_axes[3].pcolormesh(
        ts_mean["G6solar-G6sulfur"].longitude,
        ts_mean["G6solar-G6sulfur"].latitude,
        ts_mean["G6solar-G6sulfur"],
        vmin=-g6solar_g6sulfur_vmax,
        vmax=g6solar_g6sulfur_vmax,
        cmap="bwr",
        shading="auto",
        transform=ccrs.PlateCarree()
    )
    
    row_axes[3].add_feature(cfeature.LAND,facecolor="white",zorder=10)
    row_axes[3].coastlines()
    row_axes[3].set_title("G6solar − G6sulfur", fontsize=10)


# --- Shared colorbars per row, aligned with rightmost panel ---
for row_idx, im in enumerate([im0, im1, im2, im3]):
    # Rightmost axes in this row
    ax = axes[row_idx, -1]

    # Get its position in figure coordinates
    pos = ax.get_position()  # Bbox(x0, y0, x1, y1)

    # Create colorbar axes slightly to the right
    cbar_ax = fig.add_axes([pos.x1 + 0.01, pos.y0, 0.015, pos.height])  # left, bottom, width, height

    # Add colorbar
    fig.colorbar(im, cax=cbar_ax, orientation='vertical')

plt.show()


In [ ]:
print(model_meta["ts_variability"].min().values)
print(model_meta["ts_variability"].max().values)


In [ ]:
diff_sul_245 = ts_mean["G6sulfur-SSP245"]
Sig_sul_245  = np.abs(diff_sul_245) - ts_var

diff_sul_245_com = diff_sul_245.compute()
Sig_sul_245_com  = Sig_sul_245.compute()


diff_sol_245 = ts_mean["G6solar-SSP245"]
Sig_sol_245  = np.abs(diff_sol_245) - ts_var

diff_sol_245_com = diff_sol_245.compute()
Sig_sol_245_com  = Sig_sol_245.compute()


diff_sol_sul = ts_mean["G6solar-G6sulfur"]
Sig_sol_sul  = np.abs(diff_sol_sul) - ts_var   # <-- FIXED

diff_sol_sul_com = diff_sol_sul.compute()
Sig_sol_sul_com  = Sig_sol_sul.compute()





In [ ]:
#Plotting only the significant areas of SST change.

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Mask non-significant values
diff_sul_245_sig = diff_sul_245_com.where(Sig_sul_245_com > 0)
diff_sol_245_sig = diff_sol_245_com.where(Sig_sol_245_com > 0)
diff_sol_sul_sig = diff_sol_sul_com.where(Sig_sol_sul_com > 0)

fig, axes = plt.subplots(
    nrows=3,
    ncols=1,
    figsize=(14, 14),
    subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180))
)

datasets = [
    (diff_sul_245_sig, "G6sulfur − SSP245 (Only, $|\Delta_{(2071,2100)-SSP245}| > \sigma_{piControl}$)"),
    (diff_sol_245_sig, "G6solar − SSP245 (Only, $|\Delta_{(2071,2100)-SSP245}| > \sigma_{piControl}$)"),
    (diff_sol_sul_sig, "G6solar − G6sulfur (Only, $|\Delta_{(2071,2100)-SSP245}| > \sigma_{piControl}$)")
]

for ax, (da, title) in zip(axes, datasets):
    pcm = ax.pcolormesh(
        da.longitude,
        da.latitude,
        da,
        cmap="RdBu_r",
        vmin=-1,
        vmax=1,
        shading="auto",
        transform=ccrs.PlateCarree()
    )

    ax.add_feature(cfeature.LAND, facecolor="grey", zorder=10)
    ax.coastlines()
    ax.set_title(title)

# shared colorbar
cbar = fig.colorbar(
    pcm,
    ax=axes,
    orientation="vertical",
    fraction=0.03,
    pad=0.02,
    label="Δ SST (K)"
)

# Save
fig_directory = "/home/users/bidyut/20260112_Basic_Analysis"
figname = "Figure_SST_UKESM"
# # plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.svg', format="svg",transparent=True, dpi=1200)
# # !rsvg-convert -f pdf -o {fig_directory}/{figname}.pdf {fig_directory}/{figname}.svg
plt.savefig(f"{fig_directory}/{figname}.pdf", format="pdf")

plt.show()


In [ ]:
### TESTING

# diagnostic field
diff = ts_mean["G6sulfur-SSP245"]
D = np.abs(diff) - ts_var

#COMPUTE (since it was a DUSK arraw)
Dcom=D.compute()

In [ ]:
#Slicing

D_tropics = Dcom.where(
    (Dcom.latitude >= -30) & (Dcom.latitude <= 30),
    drop=True
)

In [ ]:
# Variable to plot
Dplot=Dcom

In [ ]:
#PLOTTING 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import xarray as xr
from matplotlib.colors import ListedColormap

# significance mask from your diagnostic field
sig = Dplot > 0

# encode as integers for plotting
sig_int = xr.where(sig, 1, 0)

# two-color map: 0=non-sig (red), 1=sig (blue)
cmap = ListedColormap(["red", "blue"])

fig, ax = plt.subplots(
    figsize=(14, 8),
    subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180))
)

ax.pcolormesh(
    Dplot.longitude,
    Dplot.latitude,
    sig_int,
    cmap=cmap,
    shading="auto",
    transform=ccrs.PlateCarree()
)

# land mask on top
ax.add_feature(cfeature.LAND, facecolor="white", zorder=10)
ax.coastlines()

ax.set_title(r"Significance mask from $|\Delta| - \sigma$  (blue = significant)")

plt.show()

